# 🔋 Tối Ưu Hóa Năng Lượng Microgrid - PPO (Policy Gradient)

---

## 📌 Hướng Dẫn Sử Dụng (CHỈ CẦN 3 BƯỚC)

| Bước | Hành Động | Thời Gian |
|------|-----------|------------|
| **1** | Chạy ô "📦 Cài Đặt" | ~10 giây |
| **2** | Chạy ô "🚀 Huấn Luyện" | ~60 giây |
| **3** | Chạy ô "📊 Kết Quả" | ~5 giây |

---

## 🎯 Bài Toán

**Mục tiêu:** Dạy AI quản lý năng lượng thông minh cho hệ thống microgrid

```
☀️ Năng lượng mặt trời  ─┐
🌬️ Năng lượng gió      ─┼─► 🤖 PPO Agent ─► ⚡ Cung cấp điện cho hộ gia đình
🔋 Pin lưu trữ         ─┤
⚡ Lưới điện           ─┘
```

## 🧠 PPO vs DQN - Khác biệt chính

| Đặc điểm | DQN | PPO |
|-----------|-----|-----|
| Loại | Value-based | Policy-based |
| Mạng | Q-Network | Actor-Critic |
| Output | Q-values → chọn action có Q cao nhất | Xác suất trực tiếp π(a|s) |
| Training | Off-policy (replay buffer) | On-policy (rollout buffer) |
| Ưu điểm | Đơn giản, ổn định | Linh hoạt, policy smooth |

**PPO = Proximal Policy Optimization (Schulman et al., 2017)**

---
## 📦 BƯỚC 1: Cài Đặt (Chạy 1 lần)

Ô này cài đặt thư viện và tạo môi trường microgrid + PPO Agent.

In [ ]:
#@title 📦 BƯỚC 1: CÀI ĐẶT - Chạy ô này trước

# ═══════════════════════════════════════════════════════════════════════════
# 📌 CÀI ĐẶT THƯ VIỆN
# ═══════════════════════════════════════════════════════════════════════════
!pip install gymnasium torch numpy matplotlib -q

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import random
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Thiết bị: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# ═══════════════════════════════════════════════════════════════════════════
# 🔧 CẤU HÌNH - Thay đổi các giá trị này cho bài làm riêng!
# ═══════════════════════════════════════════════════════════════════════════
SEED = 42              # 🔧 Thay đổi: 42, 123, 456, 789, 999
EPISODES = 200         # 🔧 Thay đổi: 100-500
LR_ACTOR = 3e-4        # 🔧 Thay đổi: 1e-4 - 1e-3
LR_CRITIC = 1e-3       # 🔧 Thay đổi: 5e-4 - 3e-3
CLIP_EPSILON = 0.2     # 🔧 PPO clip range: 0.1-0.3
PPO_EPOCHS = 10        # 🔧 Update epochs: 5-15
GAE_LAMBDA = 0.95      # 🔧 GAE lambda: 0.9-0.99
GAMMA = 0.99           # 🔧 Discount factor: 0.95-0.99

np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

# ═══════════════════════════════════════════════════════════════════════════
# 🌍 MÔI TRƯỜNG MICROGRID (giống bản DQN để so sánh công bằng)
# ═══════════════════════════════════════════════════════════════════════════
class MicrogridEnv:
    """Môi trường microgrid: Solar + Wind + Battery + Grid"""
    def __init__(self):
        self.battery_capacity = 100.0
        self.battery_level = 50.0
        self.current_hour = 0

    def reset(self):
        self.battery_level = 50.0
        self.current_hour = 0
        return self._get_state()

    def _get_state(self):
        hour = self.current_hour % 24
        solar = max(0, np.sin((hour - 6) * np.pi / 12)) * 50 * (0.8 + 0.4 * np.random.random())
        wind = 30 * np.random.random()
        demand = 40 + 20 * np.sin((hour - 6) * np.pi / 12) + 10 * np.random.random()
        price = 0.15 + 0.1 * (1 if 17 <= hour <= 21 else 0)
        return np.array([
            self.battery_level / self.battery_capacity,
            demand / 100, solar / 50, wind / 30, price / 0.25,
            np.sin(2 * np.pi * hour / 24),
            np.cos(2 * np.pi * hour / 24), 0.5
        ], dtype=np.float32)

    def step(self, action):
        state = self._get_state()
        demand = state[1] * 100
        solar = state[2] * 50
        wind = state[3] * 30
        price = state[4] * 0.25
        renewable = solar + wind
        grid_used = renewable_used = battery_change = 0

        if action == 0:  # Xả pin
            discharge = min(20, self.battery_level, demand)
            battery_change = -discharge
            renewable_used = min(renewable, demand - discharge)
            grid_used = max(0, demand - discharge - renewable_used)
        elif action == 1:  # Sạc từ renewable
            renewable_used = min(renewable, demand)
            grid_used = max(0, demand - renewable_used)
            excess = renewable - renewable_used
            battery_change = min(20, self.battery_capacity - self.battery_level, excess)
        elif action == 2:  # Mua từ lưới
            grid_used = demand
        elif action == 3:  # Renewable + xả pin
            renewable_used = min(renewable, demand)
            remaining = demand - renewable_used
            discharge = min(20, self.battery_level, remaining)
            battery_change = -discharge
            grid_used = max(0, remaining - discharge)
        else:  # Renewable + lưới
            renewable_used = min(renewable, demand)
            grid_used = max(0, demand - renewable_used)

        self.battery_level = np.clip(self.battery_level + battery_change, 0, self.battery_capacity)
        reward = renewable_used / 40 - 2 * grid_used / 40 * price - 0.1 * abs(battery_change) / 20
        if grid_used == 0 and 17 <= self.current_hour % 24 <= 21:
            reward += 0.5
        self.current_hour += 1
        done = self.current_hour >= 24
        return self._get_state(), reward, done, {
            'renewable_used': renewable_used, 'grid_used': grid_used, 'cost': grid_used * price
        }

# ═══════════════════════════════════════════════════════════════════════════
# 🧠 ACTOR-CRITIC NETWORK (khác DQN: có 2 heads)
# ═══════════════════════════════════════════════════════════════════════════
class ActorCritic(nn.Module):
    """
    Actor-Critic Network:
    - Actor: Output xác suất chọn mỗi action → π(a|s)
    - Critic: Output giá trị trạng thái → V(s)
    """
    def __init__(self):
        super().__init__()
        # Shared layers
        self.shared = nn.Sequential(
            nn.Linear(8, 128), nn.Tanh(),   # 🔧 Thay đổi: 64, 128, 256
            nn.Linear(128, 128), nn.Tanh(),
        )
        # Actor head → probabilities
        self.actor = nn.Sequential(nn.Linear(128, 5), nn.Softmax(dim=-1))
        # Critic head → state value
        self.critic = nn.Linear(128, 1)

    def forward(self, x):
        features = self.shared(x)
        return self.actor(features), self.critic(features)

# ═══════════════════════════════════════════════════════════════════════════
# 🤖 PPO AGENT
# ═══════════════════════════════════════════════════════════════════════════
class PPOAgent:
    """
    PPO Agent - Proximal Policy Optimization
    Khác DQN:
    - Không dùng replay buffer → dùng rollout buffer (on-policy)
    - Không dùng epsilon-greedy → sample từ policy distribution
    - Clipped objective → ngăn policy thay đổi quá lớn
    """
    def __init__(self):
        self.network = ActorCritic().to(device)
        self.optimizer = optim.Adam([
            {'params': self.network.shared.parameters(), 'lr': LR_ACTOR},
            {'params': self.network.actor.parameters(), 'lr': LR_ACTOR},
            {'params': self.network.critic.parameters(), 'lr': LR_CRITIC},
        ])
        # Rollout storage
        self.states = []
        self.actions = []
        self.log_probs = []
        self.rewards = []
        self.values = []
        self.dones = []

    def select_action(self, state, eval_mode=False):
        state_t = torch.FloatTensor(state).unsqueeze(0).to(device)
        with torch.no_grad():
            probs, value = self.network(state_t)
        if eval_mode:
            return probs.argmax().item()
        dist = Categorical(probs)
        action = dist.sample()
        # Store for PPO update
        self.states.append(state)
        self.actions.append(action.item())
        self.log_probs.append(dist.log_prob(action).item())
        self.values.append(value.item())
        return action.item()

    def store_reward(self, reward, done):
        self.rewards.append(reward)
        self.dones.append(float(done))

    def update(self):
        """PPO Update: GAE → Clipped Surrogate → Multiple epochs"""
        if len(self.states) == 0:
            return

        # 1. Compute GAE advantages
        advantages = []
        gae = 0
        values = self.values + [0]  # Bootstrap = 0 for terminal
        for t in reversed(range(len(self.rewards))):
            delta = self.rewards[t] + GAMMA * values[t+1] * (1 - self.dones[t]) - values[t]
            gae = delta + GAMMA * GAE_LAMBDA * (1 - self.dones[t]) * gae
            advantages.insert(0, gae)

        advantages = torch.FloatTensor(advantages).to(device)
        returns = advantages + torch.FloatTensor(self.values).to(device)
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # Prepare tensors
        states = torch.FloatTensor(np.array(self.states)).to(device)
        actions = torch.LongTensor(self.actions).to(device)
        old_log_probs = torch.FloatTensor(self.log_probs).to(device)

        # 2. PPO epochs
        for _ in range(PPO_EPOCHS):
            probs, values_pred = self.network(states)
            dist = Categorical(probs)
            new_log_probs = dist.log_prob(actions)
            entropy = dist.entropy()

            # Clipped surrogate objective
            ratio = torch.exp(new_log_probs - old_log_probs)
            surr1 = ratio * advantages
            surr2 = torch.clamp(ratio, 1 - CLIP_EPSILON, 1 + CLIP_EPSILON) * advantages
            policy_loss = -torch.min(surr1, surr2).mean()

            value_loss = nn.MSELoss()(values_pred.squeeze(), returns)
            loss = policy_loss + 0.5 * value_loss - 0.01 * entropy.mean()

            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self.network.parameters(), 0.5)
            self.optimizer.step()

        # Clear rollout buffer
        self.states = []
        self.actions = []
        self.log_probs = []
        self.rewards = []
        self.values = []
        self.dones = []

# Khởi tạo
env = MicrogridEnv()
agent = PPOAgent()
print("\n✅ Cài đặt hoàn tất! Chuyển sang BƯỚC 2.")
print(f"   🧠 Thuật toán: PPO (Proximal Policy Optimization)")
print(f"   📊 Clip ε: {CLIP_EPSILON}, PPO Epochs: {PPO_EPOCHS}")
print(f"   📈 LR Actor: {LR_ACTOR}, LR Critic: {LR_CRITIC}")

---
## 🚀 BƯỚC 2: Huấn Luyện AI (PPO)

Ô này huấn luyện PPO Agent.

**Khác biệt training PPO vs DQN:**
- DQN: Mỗi step → lưu replay buffer → sample batch → update
- PPO: Thu thập rollout (nhiều episodes) → tính GAE advantage → update nhiều epochs → clear buffer

In [ ]:
#@title 🚀 BƯỚC 2: HUẤN LUYỆN PPO AGENT

print("═" * 60)
print("🚀 BẮT ĐẦU HUẤN LUYỆN PPO AGENT")
print("═" * 60)
print(f"📊 Số episodes: {EPISODES}")
print(f"🎲 Random seed: {SEED}")
print(f"📈 Clip ε: {CLIP_EPSILON}, GAE λ: {GAE_LAMBDA}")
print("═" * 60)

rewards_history = []
renewable_history = []
UPDATE_EVERY = 4  # Update sau mỗi 4 episodes

for episode in range(1, EPISODES + 1):
    state = env.reset()
    total_reward = 0
    total_renewable = 0
    total_grid = 0

    for step in range(24):  # 24 giờ/ngày
        action = agent.select_action(state)
        next_state, reward, done, info = env.step(action)
        agent.store_reward(reward, done)

        state = next_state
        total_reward += reward
        total_renewable += info['renewable_used']
        total_grid += info['grid_used']

        if done:
            break

    # PPO update sau mỗi UPDATE_EVERY episodes
    if episode % UPDATE_EVERY == 0:
        agent.update()

    # Lưu history
    rewards_history.append(total_reward)
    renewable_ratio = total_renewable / (total_renewable + total_grid + 1e-6) * 100
    renewable_history.append(renewable_ratio)

    if episode % 10 == 0:
        avg_reward = np.mean(rewards_history[-10:])
        print(f"Episode {episode:3d} | Reward: {avg_reward:+7.2f} | Renewable: {renewable_ratio:.1f}%")

print("\n" + "═" * 60)
print("✅ HUẤN LUYỆN HOÀN TẤT!")
print(f"📈 Best Reward: {max(rewards_history):.2f}")
print(f"📈 Final Avg Reward: {np.mean(rewards_history[-10:]):.2f}")
print("═" * 60)
print("\n👉 Chuyển sang BƯỚC 3 để xem kết quả!")

---
## 📊 BƯỚC 3: Xem Kết Quả

Ô này hiển thị:
- 📈 Đồ thị quá trình học PPO
- 📊 So sánh PPO Agent vs Random
- 📋 Bảng số liệu kết quả

In [ ]:
#@title 📊 BƯỚC 3: XEM KẾT QUẢ - Chạy ô này để xem đồ thị

# ═══════════════════════════════════════════════════════════════════════════
# 📊 VẼ ĐỒ THỊ TRAINING
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Đồ thị 1: Reward
axes[0].plot(rewards_history, alpha=0.3, color='purple')
window = min(10, len(rewards_history))
if len(rewards_history) >= window:
    smoothed = np.convolve(rewards_history, np.ones(window)/window, mode='valid')
    axes[0].plot(range(window-1, len(rewards_history)), smoothed, color='purple', linewidth=2)
axes[0].set_title('📈 Reward Theo Episode (PPO)', fontsize=12)
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Total Reward')
axes[0].grid(True, alpha=0.3)

# Đồ thị 2: Renewable usage
axes[1].plot(renewable_history, color='green', alpha=0.3)
if len(renewable_history) >= window:
    smoothed = np.convolve(renewable_history, np.ones(window)/window, mode='valid')
    axes[1].plot(range(window-1, len(renewable_history)), smoothed, color='green', linewidth=2)
axes[1].set_title('☀️ Tỷ Lệ Năng Lượng Tái Tạo', fontsize=12)
axes[1].set_xlabel('Episode')
axes[1].set_ylabel('Renewable %')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ppo_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

# ═══════════════════════════════════════════════════════════════════════════
# 📊 SO SÁNH VỚI RANDOM BASELINE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "═" * 60)
print("📊 ĐÁNH GIÁ: PPO AGENT vs RANDOM BASELINE")
print("═" * 60)

def evaluate(agent, use_random=False, num_episodes=10):
    total_rewards, total_costs, total_renewable = [], [], []
    for _ in range(num_episodes):
        state = env.reset()
        ep_reward = ep_cost = ep_renewable = ep_total = 0
        for step in range(24):
            if use_random:
                action = random.randint(0, 4)
            else:
                action = agent.select_action(state, eval_mode=True)
            next_state, reward, done, info = env.step(action)
            ep_reward += reward
            ep_cost += info['cost']
            ep_renewable += info['renewable_used']
            ep_total += info['renewable_used'] + info['grid_used']
            state = next_state
            if done: break
        total_rewards.append(ep_reward)
        total_costs.append(ep_cost)
        total_renewable.append(ep_renewable / ep_total * 100 if ep_total > 0 else 0)
    return {'reward': np.mean(total_rewards), 'cost': np.mean(total_costs), 'renewable': np.mean(total_renewable)}

trained_results = evaluate(agent, use_random=False)
random_results = evaluate(agent, use_random=True)

reward_imp = ((trained_results['reward'] - random_results['reward']) / abs(random_results['reward'])) * 100 if random_results['reward'] != 0 else 0
cost_imp = ((random_results['cost'] - trained_results['cost']) / random_results['cost']) * 100 if random_results['cost'] != 0 else 0

print(f"\n{'Metric':<25} {'PPO Agent':>15} {'Random':>15} {'Improvement':>15}")
print("-" * 70)
print(f"{'Mean Episode Reward':<25} {trained_results['reward']:>+15.2f} {random_results['reward']:>+15.2f} {reward_imp:>+14.1f}%")
print(f"{'Daily Cost ($)':<25} {trained_results['cost']:>15.2f} {random_results['cost']:>15.2f} {cost_imp:>+14.1f}%")
print(f"{'Renewable Usage (%)':<25} {trained_results['renewable']:>14.1f}% {random_results['renewable']:>14.1f}%")

# ═══════════════════════════════════════════════════════════════════════════
# 📊 VẼ BIỂU ĐỒ SO SÁNH
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
metrics = ['Reward', 'Cost ($)', 'Renewable %']
trained_vals = [trained_results['reward'], trained_results['cost'], trained_results['renewable']]
random_vals = [random_results['reward'], random_results['cost'], random_results['renewable']]
colors = ['#9b59b6', '#e74c3c']  # Purple for PPO

for i, (metric, t_val, r_val) in enumerate(zip(metrics, trained_vals, random_vals)):
    bars = axes[i].bar(['PPO', 'Random'], [t_val, r_val], color=colors)
    axes[i].set_title(metric, fontsize=12, fontweight='bold')
    axes[i].grid(True, alpha=0.3)
    for bar, val in zip(bars, [t_val, r_val]):
        axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                     f'{val:.1f}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('ppo_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "═" * 60)
print("✅ KẾT QUẢ ĐÃ ĐƯỢC LƯU!")
print("   📁 ppo_training_curves.png")
print("   📁 ppo_comparison.png")
print("═" * 60)

print("\n📝 KẾT LUẬN:")
print(f"   ✅ PPO Agent đã học được chính sách quản lý năng lượng tối ưu")
print(f"   ✅ Cải thiện reward: {reward_imp:+.1f}% so với random")
print(f"   ✅ Giảm chi phí điện: {cost_imp:+.1f}%")
print(f"   ✅ Tỷ lệ năng lượng tái tạo: {trained_results['renewable']:.1f}%")
print(f"   🧠 Thuật toán: PPO (Schulman et al., 2017)")

---
## 📚 Giải Thích PPO Chi Tiết

### 🎯 5 Hành Động Của Agent (giống DQN)

| Action | Tên | Ý Nghĩa |
|--------|-----|--------|
| 0 | Xả pin | Dùng năng lượng trong pin để cấp điện |
| 1 | Sạc từ renewable | Lưu năng lượng mặt trời/gió vào pin |
| 2 | Mua từ lưới | Mua toàn bộ điện từ lưới điện |
| 3 | Renewable + Xả pin | Ưu tiên dùng năng lượng tái tạo, thiếu thì xả pin |
| 4 | Renewable + Lưới | Ưu tiên dùng năng lượng tái tạo, thiếu thì mua lưới |

### 🧠 PPO Học Như Thế Nào?

```
1. 🎲 Thu thập rollout: Chạy policy hiện tại, lưu (s, a, r, log_prob)
   ↓
2. 📊 Tính GAE Advantage: A_t = Σ (γλ)^l · δ_{t+l}
   ↓
3. 🔄 PPO Update (nhiều epochs):
   L = min(ratio × A, clip(ratio, 1-ε, 1+ε) × A)
   ratio = π_new(a|s) / π_old(a|s)
   ↓
4. 🗑️ Clear buffer → quay lại bước 1
```

### 🔑 Công Thức Chính

**Clipped Surrogate Objective:**
```
L_CLIP = E[min(r(θ)·A, clip(r(θ), 1-ε, 1+ε)·A)]
```
- `r(θ) = π_new / π_old` : tỷ số xác suất mới/cũ
- `A` : advantage (GAE)
- `ε` : clip range (thường 0.2)
- Clipping ngăn policy thay đổi quá lớn → training ổn định

---

## 🔧 Tùy Chỉnh Cho Bài Làm Riêng

| Sinh viên | SEED | EPISODES | LR_ACTOR | CLIP_ε | PPO_EPOCHS |
|-----------|------|----------|----------|--------|------------|
| SV1 | 42 | 200 | 3e-4 | 0.2 | 10 |
| SV2 | 123 | 300 | 1e-4 | 0.1 | 8 |
| SV3 | 456 | 150 | 5e-4 | 0.3 | 15 |
| SV4 | 789 | 250 | 2e-4 | 0.2 | 5 |
| SV5 | 999 | 400 | 4e-4 | 0.15 | 12 |

### 📚 Tài Liệu Tham Khảo
- PPO: Schulman et al., 2017 - "Proximal Policy Optimization Algorithms"
- GAE: Schulman et al., 2015 - "High-Dimensional Continuous Control Using GAE"
- Actor-Critic: Konda & Tsitsiklis, 2000 - "Actor-Critic Algorithms"